In [17]:
#  Licensed to the Apache Software Foundation (ASF) under one
#  or more contributor license agreements.  See the NOTICE file
#  distributed with this work for additional information
#  regarding copyright ownership.  The ASF licenses this file
#  to you under the Apache License, Version 2.0 (the
#  "License"); you may not use this file except in compliance
#  with the License.  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
# limitations under the License.

# Implementing Slowly Changing Dimensions (SCD Type 2 & 4) with Apache Hudi
Welcome to this deep dive into implementing two common data warehousing patterns - Slowly Changing Dimensions (SCD) Type 2 and Type 4—using Apache Hudi.

SCDs are used to handle changes to dimension data over time. Instead of simply updating a record, which would lose historical context, SCDs allow us to track changes. Hudi's powerful upsert capabilities and metadata features make it an ideal tool for this kind of workload, simplifying what would otherwise be a complex process.

In this notebook, we will cover:

**SCD Type 2:** Tracking changes by adding new rows to the dimension table.

**SCD Type 4:** Storing historical data in a separate history table.

## Setting up the Environment
First, we begin by importing our necessary libraries and starting a SparkSession configured to work with Hudi and MinIO.

In [2]:
%run utils.ipynb

Now, let's start the SparkSession. We'll give it the app name 'SCD-Types' and configure it to use our Hudi and MinIO settings.

In [3]:
spark = get_spark_session("SCD-Types")

25/09/19 16:39:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


SparkSession started with app name: SCD-Types


## Implementing SCD Type 2 with Hudi
SCD Type 2 is a method for tracking history by creating a new row for each change to a dimension record. The old row is marked as inactive, and the new row becomes the current record. This is perfect for capturing a full history of changes, such as a customer's address or a product's price.

We will add two fields to track the history:
- **end_date:** A timestamp marking when a record's version became inactive.
- **current_flag:** A boolean flag that is true for the most recent version and false for older versions.

## Data and Hudi Configuration for SCD Type 2
First, let's define our initial dataset. These records will be the starting point for our SCD Type 2 table.

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, BooleanType
from pyspark.sql.functions import col, lit, concat_ws

scd2_data = [
    ("2025-08-10 08:15:30", "uuid-001", "rider-A", "driver-X", 18.50, "new_york"),
    ("2025-08-10 09:22:10", "uuid-002", "rider-B", "driver-Y", 22.75, "san_francisco"),
    ("2025-08-10 10:05:45", "uuid-003", "rider-C", "driver-Z", 14.60, "chicago")
]
columns = ["ts", "uuid", "rider", "driver", "fare", "city"]

scd2_initial_df = spark.createDataFrame(scd2_data, columns) \
    .withColumn("effective_date", col("ts")) \
    .withColumn("end_date", lit(None).cast(StringType())) \
    .withColumn("current_flag", lit(True).cast(BooleanType())) \
    .withColumn("record_key", concat_ws("_", col("uuid"), col("ts")))

Now, let's configure our Hudi table for SCD Type 2. The key here is that our recordkey must be a unique identifier for each version of a record. To achieve this, we'll create a composite key using user_id and effective_date. We will also enable Change Data Capture to demonstrate its functionality later on.

In [5]:
scd2_table_name = "trips_scd2"
base_path = "s3a://warehouse/hudi-scd2"

scd2_hudi_conf = {
    "hoodie.table.name": scd2_table_name,
    "hoodie.datasource.write.recordkey.field": "record_key",
    "hoodie.datasource.write.table.type": "COPY_ON_WRITE",
    "hoodie.datasource.write.precombine.field": "effective_date",
    "hoodie.table.cdc.enabled": "true",
    "hoodie.datasource.write.hive_style_partitioning": "true"
}

scd2_initial_df.write.format("hudi") \
    .options(**scd2_hudi_conf) \
    .mode("overwrite") \
    .save(f"{base_path}/{scd2_table_name}")

spark.read.format("hudi").load(f"{base_path}/{scd2_table_name}").createOrReplaceTempView(scd2_table_name)

# WARNING: Unable to attach Serviceability Agent. Unable to attach even with module exceptions: [org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed.]


Let's look at the initial state of our table. You should see two records, both with current_flag set to True and a null end_date, indicating they are the most current version.

In [6]:
scd2_df = spark.sql(f"SELECT uuid, driver, fare, effective_date, end_date, current_flag FROM {scd2_table_name} ORDER BY effective_date")
display(scd2_df)

uuid,driver,fare,effective_date,end_date,current_flag
uuid-001,driver-X,18.50,2025-08-10 08:15:30,None,True
uuid-002,driver-Y,22.75,2025-08-10 09:22:10,None,True
uuid-003,driver-Z,14.60,2025-08-10 10:05:45,None,True


Now, we'll simulate a change. We'll update the city for uuid-001 to "boston" and change the fare to 20.00.

In [7]:
# Create a new DataFrame with the updated record for uuid-001
scd2_update_data = [
    ("uuid-001", "driver-X", 20.00, "2025-08-10 08:30:00", "boston")
]
scd2_update_df = spark.createDataFrame(scd2_update_data, ["uuid", "driver", "fare", "effective_date", "city"])

To implement the SCD Type 2 logic, we first need to identify and "expire" the old records. We do this by finding the records that have the same user_id but a different city in the incoming data.

In [8]:
from pyspark.sql.functions import to_timestamp, current_timestamp

# Load the existing table and join with the incoming updates
existing_df = spark.read.format("hudi").load(f"{base_path}/{scd2_table_name}")

# Find records in the existing table that need to be expired
expired_df = existing_df.alias("old") \
    .join(scd2_update_df.alias("new"), "uuid") \
    .filter((col("old.city") != col("new.city")) | (col("old.fare") != col("new.fare"))) \
    .withColumn("end_date", col("new.effective_date")) \
    .withColumn("current_flag", lit(False)) \
    .withColumn("record_key", concat_ws("_", col("old.uuid"), col("old.effective_date"))) \
    .select(
        col("old.ts"), col("old.uuid"), col("old.rider"), col("old.driver"),
        col("old.fare"), col("old.city"), col("old.effective_date"),
        col("end_date"), col("current_flag"), col("record_key")
    )

# Constructing the new version of the record with all original columns
new_version_df = existing_df.alias("old") \
    .join(scd2_update_df.alias("new"), "uuid") \
    .select(
        col("old.ts"), col("old.uuid"), col("old.rider"), col("new.driver"),
        col("new.fare"), col("new.city"), col("new.effective_date"),
        lit(None).cast(StringType()).alias("end_date"),
        lit(True).cast(BooleanType()).alias("current_flag")
    ) \
    .withColumn("record_key", concat_ws("_", col("uuid"), col("effective_date")))


# Combine the new and expired records for the upsert
scd2_upsert_df = expired_df.union(new_version_df)

Finally, we perform the upsert with our combined DataFrame. This will automatically update the old record and insert the new one, all in a single atomic transaction.

In [9]:
scd2_upsert_df.write.format("hudi") \
    .options(**scd2_hudi_conf) \
    .mode("append") \
    .save(f"{base_path}/{scd2_table_name}")

Now, let's see the result! You should see two records for uuid-001: the old one with current_flag set to False and an end_date, and a new one with the updated city and current_flag set to True.

In [10]:
spark.sql(f"REFRESH TABLE {scd2_table_name}")
scd2_final_df = spark.sql(f"SELECT uuid, driver, fare, effective_date, end_date, city, current_flag FROM {scd2_table_name} WHERE uuid = 'uuid-001' ORDER BY effective_date, current_flag DESC")
display(scd2_final_df)

uuid,driver,fare,effective_date,end_date,city,current_flag
uuid-001,driver-X,18.5,2025-08-10 08:15:30,2025-08-10 08:30:00,new_york,False
uuid-001,driver-X,20.0,2025-08-10 08:30:00,None,boston,True


## Implementing SCD Type 4 with Hudi
SCD Type 4 uses two tables: a main dimension table for the current state and a separate history table for all past versions. This keeps the main table lean and fast for queries.

## Data and Hudi Configuration for SCD Type 4
We'll use the same initial ride data for this example.

In [11]:
scd4_data = [
    ("2025-08-10 08:15:30", "uuid-001", "rider-A", "driver-X", 18.50, "new_york"),
    ("2025-08-10 09:22:10", "uuid-002", "rider-B", "driver-Y", 22.75, "san_francisco")
]
scd4_columns = ["ts", "uuid", "rider", "driver", "fare", "city"]

scd4_initial_df = spark.createDataFrame(scd4_data).toDF(*scd4_columns)

Now, let's create two Hudi tables: a main one for the current data and a history table.

In [12]:
scd4_dim_table_name = "trips_dim_scd4"
scd4_history_table_name = "trips_history_scd4"
base_path = "s3a://warehouse/hudi-scd4"

scd4_hudi_dim_conf = {
    "hoodie.table.name": scd4_dim_table_name,
    "hoodie.datasource.write.recordkey.field": "uuid",
    "hoodie.datasource.write.table.type": "COPY_ON_WRITE",
    "hoodie.datasource.write.precombine.field": "ts"
}

scd4_hudi_history_conf = {
    "hoodie.table.name": scd4_history_table_name,
    "hoodie.datasource.write.recordkey.field": "uuid",
    "hoodie.datasource.write.table.type": "COPY_ON_WRITE",
    "hoodie.datasource.write.precombine.field": "ts"
}

scd4_initial_df.write.format("hudi") \
    .options(**scd4_hudi_dim_conf) \
    .mode("overwrite") \
    .save(f"{base_path}/{scd4_dim_table_name}")

spark.read.format("hudi").load(f"{base_path}/{scd4_dim_table_name}").createOrReplaceTempView(scd4_dim_table_name)

Let's simulate a change where the city for uuid-001 is updated.

In [13]:
scd4_update_data = [
    ("2025-08-10 08:30:00", "uuid-001", "rider-A", "driver-X", 18.50, "boston")
]
scd4_update_df = spark.createDataFrame(scd4_update_data).toDF(*scd4_columns)

To implement SCD Type 4, we first capture the "before" image of the record and write it to our history table.

In [14]:
current_dim_df = spark.read.format("hudi").load(f"{base_path}/{scd4_dim_table_name}")

# Find changed records (join + filter)
changed_records_df = current_dim_df.alias("old") \
    .join(scd4_update_df.alias("new"), col("old.uuid") == col("new.uuid")) \
    .filter((col("old.driver") != col("new.driver")) | (col("old.fare") != col("new.fare")) | (col("old.city") != col("new.city"))) \
    .select("old.ts", "old.uuid", "old.rider", "old.driver", "old.fare", "old.city")

if changed_records_df.count() > 0:
    history_ready_df = changed_records_df \
        .withColumn("uuid_ts", col("uuid") + "_" + col("ts")) \
        .withColumn("operation", lit("UPDATE"))
    
    history_ready_df.write.format("hudi") \
        .options(**scd4_hudi_history_conf) \
        .mode("append") \
        .save(f"{base_path}/{scd4_history_table_name}")

Finally, we update the main dimension table with the new data.

In [15]:
scd4_update_df.write.format("hudi") \
    .options(**scd4_hudi_dim_conf) \
    .mode("append") \
    .save(f"{base_path}/{scd4_dim_table_name}")

Now, let's look at both our main dimension table and our new history table. The dimension table contains only the latest version, while the history table holds the previous version of the record.

In [16]:
# Load both tables
main_df = spark.read.format("hudi").load(f"{base_path}/{scd4_dim_table_name}")
history_df = spark.read.format("hudi").load(f"{base_path}/{scd4_history_table_name}")

print("Current Dimension Table:")
display(main_df.select("uuid", "driver", "ts", "fare", "city").orderBy("uuid"))

print("History Table:")
display(history_df.select("uuid", "driver", "ts", "uuid_ts", "fare", "city").orderBy("uuid"))

Current Dimension Table:


uuid,driver,ts,fare,city
uuid-001,driver-X,2025-08-10 08:30:00,18.50,boston
uuid-002,driver-Y,2025-08-10 09:22:10,22.75,san_francisco


History Table:


uuid,driver,ts,uuid_ts,fare,city
uuid-001,driver-X,2025-08-10 08:30:00,uuid-001_2025-08-10 08:30:00,18.5,boston
